In [ ]:
# Fine tuning llama2 model and use own data like company profile
#
# See also medium.com blog
# https://medium.com/@datatec.studio
#

# Mount google driver
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Change to google driver folder which contains datasets
# This folder will also be used to save model

%cd /content/drive/MyDrive/QLoRA_Llama2_Lab/

lab_folder = "/content/drive/MyDrive/QLoRA_Llama2_Lab/"
dataset_name = lab_folder + "data/my_company_info.jsonl"
outputs_folder = lab_folder + "llama-7-int4-dts"


/content/drive/MyDrive/QLoRA_Llama2_Lab


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install transformers datasets peft accelerate bitsandbytes trl safetensors

In [ ]:
import os
print("Replact it with HF Access Token")
os.environ["HF_HOME_TOKEN"] = "REPLACE_WITH_YOUR_HUGGING_FACE_ACCESS_TOKEN"

In [ ]:
from datasets import load_dataset
dataset = load_dataset('text', data_files=dataset_name, split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from random import randrange

print(dataset)
print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])

Dataset({
    features: ['text'],
    num_rows: 1000
})
dataset size: 1000
{'text': '{"instruction": "Give me a list of some of the most popular song from 70s Japanese Pop singer, Mariya Takeuchi", "context": "", "response": "1. Plastic Love\\n2. Stay with Me\\n3. September\\n4. Miracle Love\\n5. Yume No Tsuzuki", "category": "brainstorming"}'}


In [ ]:
import json

def format_instruction(sample):
    parsed_text = json.loads(sample['text'])
    # Get the value of 'instruction'
    instruction = parsed_text['instruction']
    response = parsed_text['response']
    return f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
{instruction}

### Response:
{response}

"""


In [ ]:
from random import randrange

print(format_instruction(dataset[randrange(len(dataset))]))

### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
What is the primary focus of your company?

### Response:
The primary focus of Dummy-Gpt2-Datatec-Studio Inc is to pioneer innovation in areas like extending human longevity, addressing environmental challenges, and revolutionizing transportation.




In [ ]:
import torch
print(torch.cuda.get_device_capability()[0])

8


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

use_flash_attention = False

# Hugging Face model id
model_id = "NousResearch/Llama-2-7b-hf"  # non-gated


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    use_cache=False,
    use_flash_attention_2=use_flash_attention,
    device_map="auto",
)
model.config.pretraining_tp = 1


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)


# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)


In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir=outputs_folder,
    num_train_epochs=3,
    per_device_train_batch_size=6 if use_flash_attention else 4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=True # disable tqdm since with packing values are in correct
)


In [ ]:
from trl import SFTTrainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=args,
)


/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:548: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictionary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(


In [ ]:
# train
trainer.train() # there will not be a progress bar since tqdm is disabled


{'loss': 1.4949, 'learning_rate': 0.0002, 'epoch': 1.01}
{'loss': 1.2941, 'learning_rate': 0.0002, 'epoch': 2.02}
{'train_runtime': 247.1847, 'train_samples_per_second': 12.137, 'train_steps_per_second': 1.517, 'train_loss': 1.3495539206045646, 'epoch': 2.07}


TrainOutput(global_step=27, training_loss=1.3495539206045646, metrics={'train_runtime': 247.1847, 'train_samples_per_second': 12.137, 'train_steps_per_second': 1.517, 'train_loss': 1.3495539206045646, 'epoch': 2.07})

In [ ]:
# save model
trainer.save_model(outputs_folder)

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

# load base LLM model and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    outputs_folder,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(outputs_folder)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
from datasets import load_dataset
from random import randrange


# Load dataset from the hub and get a sample
dataset = load_dataset('text', data_files=dataset_name, split='train')
sample = dataset[randrange(len(dataset))]
print(sample)

{'text': '{"instruction": "Can you explain your quantum computing products in more detail?", "context": "Information about my company profile.", "response": "Our quantum computing products are cutting-edge solutions that exponentially boost computational power and network speeds, ushering in a new era of technological advancement beyond traditional computing methods.", "category": "closed_qa"}'}


In [ ]:
parsed_text = json.loads(sample['text'])
# Get the value of 'instruction'
instruction = parsed_text['instruction']


In [ ]:
prompt = "What are your products?"

In [ ]:
prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
{prompt}

### Response:
"""

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.9)

#response = parsed_text['response']
print(f"Prompt:\n{prompt}\n")
print(f"Generated response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
#print(f"Ground truth:\n{response}")


Prompt:
### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
What are your products?

### Response:


Generated response:

- Hydrogel: a biocompatible, flexible, and deformable hydrogel that forms stable, mechanically robust, and deformable scaffolds.

- Micro-scaffold: an innovative, patent-pending, flexible, and deformable micro-scaffold.

- Hydrogel: a biocompatible, flexible, and deformable hydrogel that forms stable, mechanically robust, and deformable scaff
